# Integral PI (with @reduction)

## 1.1 Initialize PyCOMPSs

In [ ]:
import pycompss.interactive as ipycompss

In [ ]:
import os
if 'BINDER_SERVICE_HOST' in os.environ:
    ipycompss.start(graph=True,                # trace=True
                   project_xml='../xml/project.xml',
                    resources_xml='../xml/resources.xml')
else:
    ipycompss.start(graph=True, monitor=1000, debug=True)  # trace=True

## 1.2 Required imports

In [ ]:
from pycompss.api.api import compss_wait_on
from pycompss.api.task import task
from pycompss.api.reduction import reduction
from pycompss.api.parameter import *

## 2 Tasks Declaration

In [ ]:
import numpy as np

In [ ]:
@task(returns=float)
def calculate_area(i, num_steps, number_of_batches, step_size):
    partial_area_sum = 0
    for i in range(i, num_steps, number_of_batches):
        x = (i+0.5) * step_size
        partial_area_sum += 4 / (1 + x**2)
    return partial_area_sum

![title](pi_resources/pi_integral.png)

In [ ]:
@reduction(chunk_size="2")
@task(returns=float, batches_partial_areas=COLLECTION_IN)
def sum_reduction(batches_partial_areas):
    total_area = 0
    for partial_area in batches_partial_areas:
        total_area += partial_area
    return total_area

Run the algorithm

In [ ]:
num_steps = 100000
number_of_batches = 10

In [ ]:
step_size = 1 / num_steps

In [ ]:
batches_partial_areas = []
for i in range(number_of_batches):
    partial_area = calculate_area(i, num_steps, number_of_batches, step_size)
    batches_partial_areas.append(partial_area)

    
total_area = sum_reduction(batches_partial_areas)

Wait for all tasks to finish and gather the result

In [ ]:
total_area = compss_wait_on(total_area)

Calculate PI

In [ ]:
pi = step_size * total_area

In [ ]:
print('PI:', pi, 'Error:', abs(np.pi-pi))

In [ ]:
ipycompss.stop()